In [33]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB :
    from google.colab import drive
    drive.mount('/content/drive')
    
if IN_COLAB :
  Dataset_path = "/content/drive/MyDrive/Colab Notebooks/DataSets/"
else :
  Dataset_path = "./DataSets/"

ml_path = Dataset_path + "ml-latest/"

In [34]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
import difflib
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.linear_model import PoissonRegressor
from sklearn.linear_model import LinearRegression

In [35]:
Movies_metadata = pd.read_csv(Dataset_path+"MoviesInfo.csv")
ratings = pd.read_csv(ml_path+"ratings.csv")

### just keep rating year between 1995, 2001

In [37]:
from datetime import datetime

# strftime('%Y-%m-%d %H:%M:%S')
ratings['year_rated'] = ratings['timestamp'].apply(lambda x: int( datetime.fromtimestamp( x ).strftime('%Y') ) )

ratings = ratings[ ratings['year_rated'].between(1995, 2005)].copy()

ratings.reset_index(drop=True, inplace=True)

In [38]:
print( f"number of Ratings : { ratings.shape[0] }")
print( f"number of movies : { ratings.groupby('movieId').count().shape[0] }")
print( f"number of users : { ratings.groupby('userId').count().shape[0] }")
print( f"range of rating : ( { np.min( ratings.groupby('rating').count().index )}, {np.max( ratings.groupby('rating').count().index )})  ")

number of Ratings : 12371075
number of movies : 8912
number of users : 139541
range of rating : ( 0.5, 5.0)  


In [39]:
Movies_metadata['language'] = Movies_metadata['language'].fillna('nolang')
freq = {}

for item in Movies_metadata['language']:
    for subitem in item.split(', '):
        if (subitem in freq):
            freq[subitem] += 1
        else:
            freq[subitem] = 1
    
for key, value in freq.items():
    print ("% s : % d"%(key, value))

English :  28413
French :  4639
Italian :  2891
German :  2352
Spanish :  2943
Russian :  1718
Mandarin :  910
Dutch :  410
Cantonese :  643
Latin :  322
Algonquin :  5
Hungarian :  211
Hebrew :  317
Korean :  636
American Sign Language :  60
Persian :  199
Danish :  284
Scottish Gaelic :  22
Hawaiian :  17
Japanese :  1967
Hindi :  898
Punjabi :  39
Urdu :  79
Sicilian :  10
Arabic :  483
Yiddish :  86
Serbo-Croatian :  70
Croatian :  43
Mongolian :  11
Romanian :  164
Malay :  19
Indonesian :  58
Finnish :  402
Luxembourgish :  7
French Sign Language :  7
Sioux :  13
Macedonian :  12
Albanian :  38
Swedish :  558
Cornish :  1
Vietnamese :  88
Swahili :  38
Navajo :  10
Maori :  18
Polish :  450
Norwegian :  214
Egyptian (Ancient) :  6
Klingon :  12
Filipino :  72
British Sign Language :  3
Xhosa :  21
Zulu :  26
Esperanto :  13
Irish :  42
Apache languages :  7
Welsh :  17
Chinese :  120
Neapolitan :  13
Pawnee :  2
Greek :  281
Catalan :  59
Czech :  205
Icelandic :  72
Afrikaans : 

In [40]:
mylang = ['English' , 'French' , 'Italian','German','Spanish','Russian', 'Other']
mylangs = ['English' , 'French' , 'Italian','German','Spanish','Russian']
for i in mylang:
    Movies_metadata[i] = 0
    
for index, row in Movies_metadata.iterrows():
#     print( index, row['imdb_title_id'], row['genres'])
    for subitem in row['language'].split(', '):
        if subitem in mylangs:
            Movies_metadata.loc[index, subitem] = 1
        else:
            Movies_metadata.loc[index, "Other"] = 1
        
Movies_metadata.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41896 entries, 0 to 41895
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movieId                41896 non-null  int64  
 1   genres                 41896 non-null  object 
 2   imdb_title_id          41896 non-null  object 
 3   title                  41896 non-null  object 
 4   original_title         41896 non-null  object 
 5   year                   41896 non-null  int64  
 6   date_published         41896 non-null  object 
 7   genre                  41896 non-null  object 
 8   duration               41896 non-null  int64  
 9   country                41890 non-null  object 
 10  language               41896 non-null  object 
 11  director               41886 non-null  object 
 12  writer                 41621 non-null  object 
 13  production_company     40944 non-null  object 
 14  actors                 41877 non-null  object 
 15  de

In [41]:
pd.options.mode.chained_assignment = None
all_genres = []

for item in Movies_metadata['genres']:
    for subitem in item.split('|'):
        if (subitem not in all_genres):
            all_genres.append( subitem )

            
for i in all_genres:
    Movies_metadata[i] = 0

for index, row in Movies_metadata.iterrows():
#     print( index, row['imdb_title_id'], row['genres'])
    for subitem in row['genres'].split('|'):
        Movies_metadata.loc[index, subitem] = 1


In [42]:
Movies_metadata.columns

Index(['movieId', 'genres', 'imdb_title_id', 'title', 'original_title', 'year',
       'date_published', 'genre', 'duration', 'country', 'language',
       'director', 'writer', 'production_company', 'actors', 'description',
       'avg_vote', 'votes', 'budget', 'usa_gross_income',
       'worlwide_gross_income', 'metascore', 'reviews_from_users',
       'reviews_from_critics', 'mean_vote', 'total_votes', 'English', 'French',
       'Italian', 'German', 'Spanish', 'Russian', 'Other', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance', 'Drama',
       'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'IMAX',
       'Documentary', 'War', 'Musical', 'Western', 'Film-Noir',
       '(no genres listed)'],
      dtype='object')

In [43]:
#myobj = ['country','language','director','production_company']
#obj_Movies_metadata= Movies_metadata.select_dtypes(include=['object']).copy()

#for i in myobj:
 #   obj_Movies_metadata[i] = obj_Movies_metadata[i].astype('category')
  #  Movies_metadata[i] = obj_Movies_metadata[i].cat.codes
    
Movies_metadata = Movies_metadata.fillna(100000)
Movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41896 entries, 0 to 41895
Data columns (total 53 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movieId                41896 non-null  int64  
 1   genres                 41896 non-null  object 
 2   imdb_title_id          41896 non-null  object 
 3   title                  41896 non-null  object 
 4   original_title         41896 non-null  object 
 5   year                   41896 non-null  int64  
 6   date_published         41896 non-null  object 
 7   genre                  41896 non-null  object 
 8   duration               41896 non-null  int64  
 9   country                41896 non-null  object 
 10  language               41896 non-null  object 
 11  director               41896 non-null  object 
 12  writer                 41896 non-null  object 
 13  production_company     41896 non-null  object 
 14  actors                 41896 non-null  object 
 15  de

In [44]:
MyCol = ['movieId','year','duration','metascore','reviews_from_users','reviews_from_critics'
        ,'mean_vote',
        'Adventure',
        'Animation', 'Children', 'Comedy', 'Fantasy', 'Romance', 'Drama',
        'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'IMAX',
        'Documentary', 'War', 'Musical', 'Western', 'Film-Noir',
        '(no genres listed)']

In [45]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled = scaler.fit_transform(Movies_metadata[MyCol].drop(['movieId'],axis=1))
data_norm = pd.DataFrame(scaled,columns=(Movies_metadata[MyCol].drop(['movieId'],axis=1).columns))
data_norm= data_norm.join(Movies_metadata[MyCol]['movieId'])
                               
from scipy import stats
z_scores = stats.zscore(data_norm.drop(['movieId'],axis=1))
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
data = data_norm[filtered_entries]

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24949 entries, 2 to 41893
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  24949 non-null  float64
 1   duration              24949 non-null  float64
 2   metascore             24949 non-null  float64
 3   reviews_from_users    24949 non-null  float64
 4   reviews_from_critics  24949 non-null  float64
 5   mean_vote             24949 non-null  float64
 6   Adventure             24949 non-null  float64
 7   Animation             24949 non-null  float64
 8   Children              24949 non-null  float64
 9   Comedy                24949 non-null  float64
 10  Fantasy               24949 non-null  float64
 11  Romance               24949 non-null  float64
 12  Drama                 24949 non-null  float64
 13  Action                24949 non-null  float64
 14  Crime                 24949 non-null  float64
 15  Thriller           

In [46]:
FinalDataset = pd.merge(ratings, data, how='inner')
FinalDataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5931415 entries, 0 to 5931414
Data columns (total 31 columns):
 #   Column                Dtype  
---  ------                -----  
 0   userId                int64  
 1   movieId               int64  
 2   rating                float64
 3   timestamp             int64  
 4   year_rated            int64  
 5   year                  float64
 6   duration              float64
 7   metascore             float64
 8   reviews_from_users    float64
 9   reviews_from_critics  float64
 10  mean_vote             float64
 11  Adventure             float64
 12  Animation             float64
 13  Children              float64
 14  Comedy                float64
 15  Fantasy               float64
 16  Romance               float64
 17  Drama                 float64
 18  Action                float64
 19  Crime                 float64
 20  Thriller              float64
 21  Horror                float64
 22  Mystery               float64
 23  Sci-Fi 

In [47]:
newdata =  FinalDataset.drop(['timestamp','year_rated'],axis=1)

# Regression

In [48]:
def fit_model(model,data):
    target = data['rating'] 
    predictors = data.drop(['userId','movieId','rating'],axis=1)
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30, random_state=40)
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)
    print('MSE:',mean_squared_error(y_test, y_pred))
    print('MAE:',mean_absolute_error(y_test, y_pred))
    print('RMSE:',np.sqrt(mean_squared_error(y_test, y_pred)))

### Poisson Regression

In [49]:
model01 = PoissonRegressor()

In [50]:
fit_model(model01,newdata[newdata['userId'] == 19])

MSE: 0.45611689867611965
MAE: 0.5283438612659538
RMSE: 0.6753642710982863


### Linear Regression

In [51]:
model02 = LinearRegression()

In [52]:
fit_model(model02,newdata[newdata['userId'] == 19])

MSE: 0.5726606770803673
MAE: 0.5827887730841136
RMSE: 0.7567434684755245


In [53]:
def get_Movie_info(Movie_id):
    
    """
    Returns some basic information about a Movie given the Movie id and the metadata dataframe.
    """
    
    Movie_info = Movies_metadata[Movies_metadata['movieId'] == int(Movie_id)][['movieId', 'genres', 
                                                            'title', 'original_title', 'year', 'duration','mean_vote']]
    return Movie_info


### Test Generate_Recommendation

In [54]:
def test_generate_recommendation(model,data):
    frames = pd.DataFrame()
    movie_all  = data.movieId.unique().tolist()
    for i in movie_all:
        mask = data[data.movieId.isin([i])]
        rat = mask['rating'].tolist()
        mask = mask.drop(['userId','movieId','rating'],axis=1)
        pred = model.predict(mask)
        tmp = get_Movie_info(i)
        tmp['pred_rating'] = pred
        tmp['rating'] = rat
        frames = frames.append(tmp, ignore_index=True)

    return frames

In [55]:
test_generate_recommendation(model01,newdata[newdata['userId'] == 19])

,movieId,genres,title,original_title,year,duration,mean_vote,pred_rating,rating
0,640,Drama|Thriller,Diabolique,Diabolique,1996,107,5.5,3.202491,3.0
1,1321,Comedy|Horror|Thriller,Un lupo mannaro americano a Londra,An American Werewolf in London,1981,97,7.6,3.202101,3.0
2,11,Comedy|Drama|Romance,Il presidente - Una storia d'amore,The American President,1995,114,7.0,3.544157,4.0
3,23,Action|Crime|Thriller,Assassins,Assassins,1995,132,6.4,3.389361,4.0
4,31,Drama,Pensieri pericolosi,Dangerous Minds,1995,99,6.7,3.305722,3.0
...,...,...,...,...,...,...,...,...,...
128,1135,Comedy,Soldato Giulia agli ordini,Private Benjamin,1980,109,6.3,3.227273,2.0
129,1170,Action,Lotta estrema,Best of the Best 3: No Turning Back,1995,90,5.1,2.995604,3.0
130,1218,Action|Crime|Drama|Thriller,The Killer,Dip huet seung hung,1989,111,7.9,3.562700,4.0
131,1347,Horror|Thriller,Nightmare - Dal profondo della notte,A Nightmare on Elm Street,1984,91,7.6,3.116353,3.0


In [56]:
test_generate_recommendation(model02,newdata[newdata['userId'] == 19])

,movieId,genres,title,original_title,year,duration,mean_vote,pred_rating,rating
0,640,Drama|Thriller,Diabolique,Diabolique,1996,107,5.5,3.330265,3.0
1,1321,Comedy|Horror|Thriller,Un lupo mannaro americano a Londra,An American Werewolf in London,1981,97,7.6,3.236847,3.0
2,11,Comedy|Drama|Romance,Il presidente - Una storia d'amore,The American President,1995,114,7.0,3.650376,4.0
3,23,Action|Crime|Thriller,Assassins,Assassins,1995,132,6.4,3.355786,4.0
4,31,Drama,Pensieri pericolosi,Dangerous Minds,1995,99,6.7,3.218140,3.0
...,...,...,...,...,...,...,...,...,...
128,1135,Comedy,Soldato Giulia agli ordini,Private Benjamin,1980,109,6.3,2.919876,2.0
129,1170,Action,Lotta estrema,Best of the Best 3: No Turning Back,1995,90,5.1,2.976546,3.0
130,1218,Action|Crime|Drama|Thriller,The Killer,Dip huet seung hung,1989,111,7.9,3.319820,4.0
131,1347,Horror|Thriller,Nightmare - Dal profondo della notte,A Nightmare on Elm Street,1984,91,7.6,2.987156,3.0


### Generate_Recommendation

In [57]:
def generate_recommendation(model,data,user_id):
    
    if user_id not in data.userId.to_list():
        print('user does not exist!')
    
    movie_seen = data[data.userId==user_id].movieId.unique().tolist()
    movie_all  = data.movieId.unique().tolist()
    
    print(f"user({user_id}) see {len(movie_seen)} movie from {len(movie_all)} movie")
    
    Movies_ID = list(set(movie_all)-set(movie_seen))
    random.shuffle(Movies_ID)
        
    data = data.drop(['userId','rating'],axis=1)
    data = data.drop_duplicates(subset=['movieId'])
    frames = pd.DataFrame()
    
    for i in movie_all:
        mask = data[data.movieId.isin([i])]
        mask = mask.drop(['movieId'],axis=1)
        pred = model.predict(mask)
        tmp = get_Movie_info(i)
        tmp['pred_rating'] = pred
        frames = frames.append(tmp, ignore_index=True)

    return frames

In [58]:
generate_recommendation(model01,newdata,19)

user(19) see 133 movie from 5262 movie


,movieId,genres,title,original_title,year,duration,mean_vote,pred_rating
0,640,Drama|Thriller,Diabolique,Diabolique,1996,107,5.5,3.202491
1,960,Crime|Drama,Infernale avventura (L'),Angel on My Shoulder,1946,100,7.0,3.291799
2,1321,Comedy|Horror|Thriller,Un lupo mannaro americano a Londra,An American Werewolf in London,1981,97,7.6,3.202101
3,1825,Comedy|Drama,The Players Club,The Players Club,1998,104,6.2,3.317560
4,1985,Horror,Halloween 4 - Il ritorno di Michael Myers,Halloween 4: The Return of Michael Myers,1988,88,6.2,2.995809
...,...,...,...,...,...,...,...,...
5257,34526,Action|Drama|Thriller,Sequestro lampo,Secuestro express,2005,90,6.9,3.312402
5258,30905,Action|Drama,Bomb the System,Bomb the System,2002,91,7.0,3.354711
5259,33160,Comedy,Death of a Dynasty,Death of a Dynasty,2003,92,4.7,2.969919
5260,8536,Drama|Thriller,Quando verrà la pioggia,The Intended,2002,110,6.2,3.317448


In [59]:
generate_recommendation(model02,newdata,19)

user(19) see 133 movie from 5262 movie


,movieId,genres,title,original_title,year,duration,mean_vote,pred_rating
0,640,Drama|Thriller,Diabolique,Diabolique,1996,107,5.5,3.330265
1,960,Crime|Drama,Infernale avventura (L'),Angel on My Shoulder,1946,100,7.0,2.448550
2,1321,Comedy|Horror|Thriller,Un lupo mannaro americano a Londra,An American Werewolf in London,1981,97,7.6,3.236847
3,1825,Comedy|Drama,The Players Club,The Players Club,1998,104,6.2,3.479653
4,1985,Horror,Halloween 4 - Il ritorno di Michael Myers,Halloween 4: The Return of Michael Myers,1988,88,6.2,2.869078
...,...,...,...,...,...,...,...,...
5257,34526,Action|Drama|Thriller,Sequestro lampo,Secuestro express,2005,90,6.9,3.263275
5258,30905,Action|Drama,Bomb the System,Bomb the System,2002,91,7.0,3.205781
5259,33160,Comedy,Death of a Dynasty,Death of a Dynasty,2003,92,4.7,2.991181
5260,8536,Drama|Thriller,Quando verrà la pioggia,The Intended,2002,110,6.2,3.200985
